In [ ]:
# if you are using .DCM files
!pip install pydicom

AUGMENT YOUR DATA

In [ ]:
import pydicom
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def augment(dicom_path, output_path):
  dicom_data = pydicom.dcmread(dicom_path)
  original_filename, extension = os.path.splitext(os.path.basename(dicom_path))
  image = dicom_data.pixel_array
  image = np.expand_dims(image, axis=-1)
  datagen = ImageDataGenerator(
      vertical_flip=True
      # horizontal_flip=True
  )
  augmented_image = datagen.apply_transform(image, transform_parameters={
            # "flip_horizontal": True
            "flip_vertical": True
        })
  augmented_image = np.squeeze(augmented_image, axis=-1)
  dicom_data.PixelData = augmented_image.tobytes()
  new_filename = original_filename + "_v" + extension
  new_dicom_file_path = os.path.join(output_path, new_filename)
  dicom_data.save_as(new_dicom_file_path)
  print(f"Saved augmented image as {output_path}")
input_directory = "/path/to/your_data"
output_directory = "/path/to/augmented_data"
for filename in os.listdir(input_directory):
    if filename.endswith(".DCM"):
      dicom_path = os.path.join(input_directory, filename)
      augment(dicom_path, output_directory)

ORGANIZE YOUR DATA FOLDER

In [ ]:
import os
import pydicom
source_directory = '/content/drive/MyDrive/decayed_part/premolar'
output_directory = '/content/drive/MyDrive/decayed_part/premolar'
dicom_files = [os.path.join(source_directory, filename) for filename in os.listdir(source_directory) if filename.endswith('.DCM')]
c = 0
for dicom_file in dicom_files:
  ds = pydicom.dcmread(dicom_file)
  # c += 1
  file_name = os.path.splitext(os.path.basename(dicom_file))[0]
  new_name = file_name + '.DCM'
  new_filename = os.path.join(output_directory, new_name)
  ds.save_as(new_filename)
# print(c)

APPLYING ROTATION AUGMENTATION AND FILTERS TO IMAGES

In [ ]:
import os
import numpy as np
import pydicom
from PIL import Image

input_folder_path = '/path/to/data'
output_folder_path = '/path/to/filtered_rotated_data'

def rotate_and_save_dicom(input_path, output_path, angle):
  dicom_data = pydicom.dcmread(input_path)
  filename = os.path.basename(input_path)
  ribbon_width = int(0.3 * dicom_data.pixel_array.shape[1])
  ribbon_position = dicom_data.pixel_array.shape[1] // 2 - ribbon_width // 2
  dicom_data.pixel_array[:, ribbon_position:ribbon_position + ribbon_width] = 0
  height, width = dicom_data.pixel_array.shape
  black_region_height = int(0.3 * height)
  if 'vertical' in filename or 'vertical' in filename:
    dicom_data.pixel_array[:black_region_height, :] = 0
  else:
    dicom_data.pixel_array[-black_region_height:, :] = 0
  pixel_array = dicom_data.pixel_array.astype(np.float32)
  pixel_array = pixel_array.astype(np.uint8)
  image = Image.fromarray(pixel_array)
  rotated_image = image.rotate(angle, resample=Image.BICUBIC, expand=True)
  rotated_pixel_array = np.array(rotated_image)
  dicom_data.PixelData = rotated_pixel_array.tobytes()
  dicom_data.Rows, dicom_data.Columns = rotated_pixel_array.shape
  dicom_data.save_as(output_path)
for filename in os.listdir(input_folder_path):
  if filename.endswith('.DCM'):
    input_file_path = os.path.join(input_folder_path, filename)
    output_file_path_20 = os.path.join(output_folder_path, f'rotated_p20_{filename}')
    rotate_and_save_dicom(input_file_path, output_file_path_20, 20)
    output_file_path_minus_20 = os.path.join(output_folder_path, f'rotated_m20_{filename}')
    rotate_and_save_dicom(input_file_path, output_file_path_minus_20, -20)

print("Rotation and saving completed.")

LOAD DATA

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np
import cv2
from PIL import Image
from skimage import exposure
path = "/path/to/data"
images = []
labels = []
for filename in os.listdir(path):
    ds = pydicom.read_file(os.path.join(path, filename))
    if 'decayed' in filename:
      labels.append(1)
    elif 'non_decayed' in filename:
      labels.append(2)
    pixel_array = ds.pixel_array.astype(np.uint8)
    ribbon_width = int(0.3 * pixel_array.shape[1])
    ribbon_position = pixel_array.shape[1] // 2 - ribbon_width // 2
    pixel_array[:, ribbon_position:ribbon_position + ribbon_width] = 0
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
    enhanced_image = clahe.apply(pixel_array)
    alpha = 1.5
    result_image = cv2.convertScaleAbs(enhanced_image, alpha=alpha, beta=0)
    height, width = result_image.shape
    upper_half = result_image[:height//3, :]
    lower_half = result_image[height//3 * 2:, :]
    if 'vertical' in filename or 'horizontal_and_vertical' in filename:
      upper_half[:, :] = 0
    else:
      lower_half[:, :] = 0
    image = Image.fromarray(result_image)
    size = (224 ,224)
    image = image.resize(size)
    # plt.imshow(image, cmap="gray")
    # plt.axis('off')
    # plt.show()
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = preprocess_input(image)
    images.append(image)

LOAD ALREADY FILTERED AND ROTATED DATA

In [ ]:
path = "/content/drive/MyDrive/decayed_part/rotated_molar"
for filename in os.listdir(path):
    ds = pydicom.read_file(os.path.join(path, filename))
    if 'ndm' in filename:
      labels.append(1)
    else:
      labels.append(2)
    pixel_array = ds.pixel_array.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
    enhanced_image = clahe.apply(pixel_array)
    alpha = 1.5
    result_image = cv2.convertScaleAbs(enhanced_image, alpha=alpha, beta=0)
    image = Image.fromarray(result_image)
    size = (224 ,224)
    image = image.resize(size)
    # plt.imshow(image, cmap="gray")
    # plt.axis('off')
    # plt.show()
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = preprocess_input(image)
    images.append(image)

In [ ]:
images = np.array(images)
images = np.stack(images, axis=0)
print(len(labels))
print(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(labels)

labels = label_encoder.transform(labels)

labels.shape

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.18, random_state=23, stratify=labels)

print("Train data shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation data shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
# print("Test data shape:", test_images.shape)
# print("Test labels shape:", test_labels.shape)

TRAIN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2, l2, l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, AveragePooling2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

model = keras.Sequential([
        base_model,
        # layers.BatchNormalization(),
        # layers.AveragePooling2D(pool_size=(2, 2), padding='same'),
        layers.Flatten(),
        layers.Dense(512, activation='relu', kernel_regularizer=l2(0.1)),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', kernel_regularizer=l2(0.1)),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')])
# freeze layers
for layer in base_model.layers:
  layer.trainable = False
# for layer in base_model.layers[-20:]:
#   layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0008), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=128, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/decayed_model.h5')

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
# plt.plot(epochs, train_loss, 'b-', label='Training Loss')
# plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()

FINE TUNE

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
# model = load_model('/path/to/decayed_model.h5')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.000008), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=128, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/finetuned_decayed_model.h5')

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()

GET YOUR MODEL'S RESULTS

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# model = load_model('/content/drive/MyDrive/decayed_part/resnet50_ribbon_finetuned_decayed_premolar_model.h5')
test_predictions = model.predict(val_images)
predicted_labels = np.round(test_predictions).astype(int).flatten()
true_labels = val_labels.flatten()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)
# Calculate precision
precision = precision_score(true_labels, predicted_labels)
print("Precision:", precision)

# Calculate recall
recall = recall_score(true_labels, predicted_labels)
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(true_labels, predicted_labels)
print("F1-score:", f1)

# Assuming you have predicted probabilities for binary classification
predicted_probabilities = test_predictions[:, 0]  # Extract probabilities for positive class

# Calculate AUC
auc = roc_auc_score(true_labels, predicted_probabilities)
print("AUC:", auc)